# utility.py

In [16]:
# My Utility : auxiliars functions
# Solo usa: pandas y numpy

import pandas as pd
import numpy  as np

# ------------------------------------------------------------
# Helpers internos
# ------------------------------------------------------------
def _embed_indices(N, d, tau):
    """Matriz (M x d) de índices para embedding con retardo tau.
       M = N - (d-1)*tau; si M<=0 retorna None."""
    M = int(N - (d - 1) * tau)
    if M <= 0:
        return None
    base  = np.arange(M)[:, None]           # (M x 1)
    steps = (np.arange(d)[None, :] * tau)   # (1 x d)
    return base + steps                     # (M x d)

def _shannon_entropy(p):
    """Entropía de Shannon (nats). Ignora p<=0 por convención."""
    p = p[p > 0]
    if p.size == 0:
        return 0.0
    return float(-(p * np.log(p)).sum())

def _quantize_dispersion_minmax(x, c):
    """Min–max → [0,1]; cuantiza a 1..c con floor(c*Xi+0.5) y clamp."""
    x = np.asarray(x, dtype=float).ravel()
    if x.size == 0:
        return np.zeros(0, dtype=int)
    xmin = np.nanmin(x); xmax = np.nanmax(x)
    if (not np.isfinite(xmin)) or (not np.isfinite(xmax)) or xmax == xmin:
        Xi = np.zeros_like(x)
    else:
        Xi = (x - xmin) / (xmax - xmin)
    Yi = np.floor(c * Xi + 0.5).astype(int)  # cuantización
    Yi[Yi < 1] = 1
    Yi[Yi > c] = c
    return Yi

def _dispersion_histogram(x, d, tau, c):
    """
    Histograma de patrones de dispersión (K=c^d bins) con codificación
    base-c CONSISTENTE: pesos ascendentes [1, c, c^2, ...].
    Devuelve (p, K) con p normalizado.
    """
    x = np.asarray(x, dtype=float).ravel()
    if x.size == 0 or d < 1 or tau < 1 or c < 2:
        return None, None

    idx = _embed_indices(x.size, d, tau)
    if idx is None:
        return None, None

    # 1) cuantización 1..c
    Y = _quantize_dispersion_minmax(x, c)

    # 2) embedding y 3) codificación base-c (Yi-1)
    emb   = Y[idx]                             # (M x d) con valores 1..c
    K     = int(c ** d)
    cpows = (c ** np.arange(d)).astype(int)    # [1, c, c^2, ...]  (clave p/consistencia)
    codes = ((emb - 1) * cpows).sum(axis=1)    # 0..K-1

    # 4) histograma → p
    counts = np.bincount(codes, minlength=K)
    total  = counts.sum()
    if total == 0:
        return None, None
    p = counts / total
    return p.astype(float), K

# ------------------------------------------------------------
# CResidual-Dispersion Entropy (CRDE) — normalizada [0,1]
# Firma EXACTA de la plantilla
# ------------------------------------------------------------
def entropy_dispersion(x, d, tau, c):
    """
    Dispersion Entropy (Shannon) normalizada en [0,1].
    """
    # usa SIEMPRE el mismo histograma/codificación que el resto
    p, K = _dispersion_histogram(x, d, tau, c)
    if p is None:
        return np.nan
    H = _shannon_entropy(p)
    
    return float(H / np.log(K)) if K > 1 else np.nan

def _entropy_dispersion_cr(x, d, tau, c):
    """
    CR Dispersion Entropy (residual), normalizada en [0,1] (versión opcional).
    """
    p, K = _dispersion_histogram(x, d, tau, c)
    if p is None:
        return np.nan
    S = np.cumsum(p[::-1])[::-1]
    Spos = S[S > 0]
    H = float(-(Spos * np.log(Spos)).sum())
    k = np.arange(K)
    Sunif = (K - k) / K
    Hmax = float(-(Sunif[Sunif > 0] * np.log(Sunif[Sunif > 0])).sum())
    return float(H / Hmax) if Hmax > 0 else np.nan


# ------------------------------------------------------------
# Permutation Entropy (PE) — normalizada [0,1]
# 
# ------------------------------------------------------------
def entropy_permuta(x, m, tau):
    """
    Permutation Entropy normalizada en [0,1] sobre la serie x.
    Pasos: embedding (m,tau) → patrón ordinal estable → histograma (m!) →
           Shannon / log(m!)
    """
    x = np.asarray(x, dtype=float).ravel()
    if x.size == 0 or m < 2 or tau < 1:
        return np.nan

    idx = _embed_indices(x.size, m, tau)
    if idx is None:
        return np.nan

    emb = x[idx]                       # (M x m)
    M   = emb.shape[0]
    codes = np.empty(M, dtype=int)

    # tie-break estable: por valores y luego por índice
    idx_local = np.arange(m)
    for i in range(M):
        row  = emb[i]
        perm = np.lexsort((idx_local, row))  # ascendente y estable

        # Código tipo Lehmer (simple)
        code = 0
        for a in range(m):
            smaller = 0
            for b in range(a+1, m):
                if perm[b] < perm[a]:
                    smaller += 1
            # multiplicador = (m-1-a)!
            mult = 1
            for t in range(2, m - a):
                mult *= t
            code += smaller * mult
        codes[i] = code

    # m! bins
    K = 1
    for t in range(2, m+1):
        K *= t

    counts = np.bincount(codes, minlength=K)
    total  = counts.sum()
    if total == 0:
        return np.nan
    p = counts / total

    H = _shannon_entropy(p)
    return float(H / np.log(K)) if K > 1 else np.nan

def entropy_mde(x, d, tau, c, S_max):
    """
    Multi-Scale Dispersion Entropy (MDE)
    x: serie temporal (array 1D)
    d: dimension embedding
    tau: factor de retardo
    c: número de símbolos
    S_max: número máximo de escalas
    Devuelve: array de MDE para escalas 1 a S_max
    """
    N = len(x)
    mde_values = np.zeros(S_max)

    for scale in range(1, S_max + 1):
        # Paso 1: Dividir en ventanas de tamaño scale
        window_size = scale
        num_windows = N // window_size

        if num_windows == 0:
            mde_values[scale-1] = np.nan
            continue

        # Calcular promedio de cada ventana
        y = np.zeros(num_windows)
        for j in range(num_windows):
            start = j * window_size
            end = (j + 1) * window_size
            y[j] = np.mean(x[start:end])

        # Paso 2: Calcular entropía de dispersión para y
        mde_values[scale-1] = entropy_dispersion(y, d, tau, c)

    return mde_values

def entropy_emde(x, d, tau, c, S_max):
    """
    Enhanced Multi-Scale Dispersion Entropy (eMDE)
    x: serie temporal (array 1D)
    d: dimension embedding
    tau: factor de retardo
    c: número de símbolos
    S_max: número máximo de escalas
    Devuelve: array de eMDE para escalas 1 a S_max
    """
    N = len(x)
    emde_values = np.zeros(S_max)

    # Escala 1: entropía de la serie original
    emde_values[0] = entropy_dispersion(x, d, tau, c)

    for scale in range(2, S_max + 1):
        # Paso 1: Generar sub-series
        avg_entropy = 0.0
        for k in range(1, scale + 1):
            # Sub-serie u_k = x[k:N]
            u_k = x[k-1:]  # Ajuste de índice para 0-based

            # Paso 2: Segmentar sub-serie
            window_size = scale
            num_windows = len(u_k) // window_size

            if num_windows == 0:
                continue

            z = np.zeros(num_windows)
            for j in range(num_windows):
                start = j * window_size
                end = (j + 1) * window_size
                z[j] = np.mean(u_k[start:end])

            # Calcular entropía para esta sub-serie
            E_k = entropy_dispersion(z, d, tau, c)
            avg_entropy += E_k

        # Promedio de entropías para esta escala
        emde_values[scale-1] = avg_entropy / scale

    return emde_values

def entropy_mpe(x, m, tau, S_max):
    """
    Multi-Scale Permutation Entropy (MPE)
    x: serie temporal (array 1D)
    m: dimension embedding
    tau: factor de retardo
    S_max: número máximo de escalas
    Devuelve: array de MPE para escalas 1 a S_max
    """
    N = len(x)
    mpe_values = np.zeros(S_max)

    for scale in range(1, S_max + 1):
        # Paso 1: Dividir en ventanas de tamaño scale
        window_size = scale
        num_windows = N // window_size

        if num_windows == 0:
            mpe_values[scale-1] = np.nan
            continue

        # Calcular promedio de cada ventana
        y = np.zeros(num_windows)
        for j in range(num_windows):
            start = j * window_size
            end = (j + 1) * window_size
            y[j] = np.mean(x[start:end])

        # Paso 2: Calcular entropía de permutación para y
        mpe_values[scale-1] = entropy_permuta(y, m, tau)

    return mpe_values

def entropy_empe(x, m, tau, S_max):
    """
    Enhanced Multi-Scale Permutation Entropy (eMPE)
    x: serie temporal (array 1D)
    m: dimension embedding
    tau: factor de retardo
    S_max: número máximo de escalas
    Devuelve: array de eMPE para escalas 1 a S_max
    """
    N = len(x)
    empe_values = np.zeros(S_max)

    # Escala 1: entropía de la serie original
    empe_values[0] = entropy_permuta(x, m, tau)

    for scale in range(2, S_max + 1):
        # Paso 1: Generar sub-series
        avg_entropy = 0.0
        for k in range(1, scale + 1):
            # Sub-serie u_k = x[k:N]
            u_k = x[k-1:]  # Ajuste de índice para 0-based

            # Paso 2: Segmentar sub-serie
            window_size = scale
            num_windows = len(u_k) // window_size

            if num_windows == 0:
                continue

            z = np.zeros(num_windows)
            for j in range(num_windows):
                start = j * window_size
                end = (j + 1) * window_size
                z[j] = np.mean(u_k[start:end])

            # Calcular entropía para esta sub-serie
            E_k = entropy_permuta(z, m, tau)
            avg_entropy += E_k

        # Promedio de entropías para esta escala
        empe_values[scale-1] = avg_entropy / scale

    return empe_values


# ppr.py


In [32]:
"""
================================================================================
REQUERIMIENTOS FUNCIONALES Y NO FUNCIONALES: ppr.py
(Etapa #1: Pre-Procesamiento de la Data)
================================================================================

--- RF-001: cargar_datos ---
| Campo               | Descripción                                                                 |
|---------------------|-----------------------------------------------------------------------------|
| **Descripción**     | Carga Class1.csv a Class4.csv (4-muestras de N=12000 valores).             |
|                     | Notación: 𝑋 ∈ ℜ^(N,4), N=12000.                                            |
| **Entradas**        | - Rutas a Class1.csv, Class2.csv, Class3.csv, Class4.csv.                   |
| **Salidas**         | - Matriz 𝑋 ∈ ℜ^(12000,4).                                                  |
| **Parámetros**      | - rutas: Lista de rutas a archivos CSV.                                     |
| **Requerimientos NF**| [RNF-001] Validar 12000 filas por archivo.                                |
|                     | [RNF-002] Validar ausencia de valores NaN.                                  |
| **Ejemplo**         | X = cargar_datos(["Class1.csv", "Class2.csv", "Class3.csv", "Class4.csv"])  |
| **Notas**           | - Usar pandas: pd.read_csv(..., header=None).                               |

--- RF-002: aplicar_diferencia_finita ---
| Campo               | Descripción                                                                 |
|---------------------|-----------------------------------------------------------------------------|
| **Descripción**     | Aplica diferencias finitas: 𝑥(n) = 𝑥(n) − 𝑥(n−1), n=2,...,N.                 |
| **Entradas**        | - Matriz 𝑋 ∈ ℜ^(12000,4).                                                  |
| **Salidas**         | - Matriz 𝑋_diff ∈ ℜ^(11999,4).                                             |
| **Parámetros**      | - X: Matriz de tamaño (12000,4).                                           |
| **Requerimientos NF**| [RNF-003] Usar np.diff(X, axis=0).                                         |
|                     | [RNF-004] Validar salida con 11999 filas.                                  |
| **Ejemplo**         | X_diff = aplicar_diferencia_finita(X)                                       |
| **Notas**           | - La primera fila se pierde al aplicar la diferencia.                       |

--- RF-003: segmentar_muestras ---
| Campo               | Descripción                                                                 |
|---------------------|-----------------------------------------------------------------------------|
| **Descripción**     | Segmenta muestras con longitud 𝑙𝐹 ∈ {1000,1200,1600}.                        |
|                     | Notación: 𝑥_i ∈ ℜ^(𝑙𝐹, 𝑛𝐹), 𝑛𝐹 = N/𝑙𝐹.                                        |
| **Entradas**        | - Matriz 𝑋_diff ∈ ℜ^(11999,4).                                             |
|                     | - 𝑙𝐹: Longitud del segmento.                                                 |
| **Salidas**         | - Lista de matrices segmentadas.                                             |
| **Parámetros**      | - X_diff: Matriz de tamaño (11999,4).                                       |
|                     | - lF: Longitud del segmento (1000, 1200, o 1600).                            |
| **Requerimientos NF**| [RNF-005] Validar que 11999 sea divisible por 𝑙𝐹.                           |
|                     | [RNF-006] Segmentar por filas, sin solapamiento.                             |
| **Ejemplo**         | segmentos = segmentar_muestras(X_diff, lF=1000)                              |
| **Notas**           | - Usar np.reshape.                                                           |

--- RF-004: calcular_entropias ---
| Campo               | Descripción                                                                 |
|---------------------|-----------------------------------------------------------------------------|
| **Descripción**     | Calcula entropías multi-escala (MDE, eMDE, MPE, eMPE) por segmento.         |
| **Entradas**        | - Lista de segmentos.                                                       |
|                     | - 𝑑, 𝜏, 𝑐, 𝑆_max: Parámetros de entropía.                                    |
| **Salidas**         | - DataFrame de entropías concatenadas (dClases.csv).                        |
| **Parámetros**      | - segmentos: Lista de matrices segmentadas.                                 |
|                     | - d: Dimensión embedding.                                                   |
|                     | - tau: Factor de retardo.                                                   |
|                     | - c: Número de símbolos.                                                    |
|                     | - Smax: Número máximo de escalas.                                           |
| **Requerimientos NF**| [RNF-007] Implementar fórmulas según slides del profesor.                  |
|                     | [RNF-008] Validar parámetros enteros positivos.                             |
| **Ejemplo**         | dClases = calcular_entropias(segmentos, d=2, tau=1, c=3, Smax=10)           |
| **Notas**           | - Concatenar los 4 datasets de entropías.                                   |

--- RF-005: generar_etiquetas ---
| Campo               | Descripción                                                                 |
|---------------------|-----------------------------------------------------------------------------|
| **Descripción**     | Crea etiquetas binarias para 4 clases: (1 0 0 0), (0 1 0 0), etc.           |
| **Entradas**        | - Número de muestras por clase.                                             |
| **Salidas**         | - DataFrame de etiquetas (dLabel.csv).                                      |
| **Parámetros**      | - n_muestras_por_clase: Número de muestras.                                 |
| **Requerimientos NF**| [RNF-009] Validar coincidencia de filas con segmentos.                     |
| **Ejemplo**         | dLabel = generar_etiquetas(n_muestras_por_clase=30000)                      |
| **Notas**           | - Usar one-hot encoding.                                                    |

--- RF-006: guardar_configuracion ---
| Campo               | Descripción                                                                 |
|---------------------|-----------------------------------------------------------------------------|
| **Descripción**     | Guarda configuración en conf_ppr.csv (6 líneas).                            |
| **Entradas**        | - tipo_entropia, lF, d, tau, c, Smax.                                       |
| **Salidas**         | - Archivo conf_ppr.csv.                                                     |
| **Parámetros**      | - tipo_entropia: 1-4 (MDE, eMDE, MPE, eMPE).                                |
|                     | - lF, d, tau, c, Smax: Parámetros numéricos.                                |
| **Requerimientos NF**| [RNF-010] Validar parámetros enteros positivos.                            |
| **Ejemplo**         | guardar_configuracion(1, 1000, 2, 1, 3, 10)                                 |
| **Notas**           | - Formato: 1 línea por parámetro.                                           |
"""

"""
ppr.py - Preprocesamiento de datos para clasificación con entropías multi-escala

Este módulo implementa las funciones necesarias para:
1. Cargar y preprocesar datos de las 4 clases
2. Calcular diferencias finitas
3. Segmentar los datos
4. Calcular entropías multi-escala (MDE, eMDE, MPE, eMPE)
5. Generar etiquetas para clasificación

El archivo está diseñado para trabajar con configuraciones externas y no contiene
valores default fijos, permitiendo optimización de parámetros desde main.py
"""
# ppr.py
"""
ppr.py - Preprocesamiento de datos para clasificación con entropías multi-escala
Este módulo implementa las funciones necesarias para:
1. Cargar y preprocesar datos de las 4 clases
2. Calcular diferencias finitas
3. Segmentar los datos
4. Calcular entropías multi-escala (MDE, eMDE, MPE, eMPE)
5. Generar etiquetas para clasificación
El archivo está diseñado para trabajar con configuraciones externas y no contiene
valores default fijos, permitiendo optimización de parámetros desde main.py
"""
import os
import numpy as np
import pandas as pd
#from utility import (entropy_dispersion, entropy_permuta,
#                    entropy_mde, entropy_emde, entropy_mpe, entropy_empe)
# Constantes de rutas
DATA_DIR = "data"
CONF_DIR = "config"
CONF_TEMP = os.path.join(CONF_DIR, "conf_temp.csv")
CONF_OPTIMO = os.path.join(CONF_DIR, "conf_optimo.csv")

def cargar_datos(rutas_classes):
    """
    Carga los datos de las 4 clases desde archivos CSV
    Returns:
        numpy.ndarray: Matriz X de tamaño (120000, 4)
    """
    matrices = []
    for ruta in rutas_classes:
        print(f"[INFO] Cargando archivo: {ruta}")
        df = pd.read_csv(ruta, header=None)
        print(f"[DEBUG] {ruta} → shape = {df.shape}")
        matriz = df.values.astype(float)
        if matriz.ndim == 1:
            matriz = matriz[:, None]
        # Seleccionar solo una columna por clase (ej. columna 0)
        matrices.append(matriz[:, [0]])

    # Validación de dimensiones
    for i, matriz in enumerate(matrices):
        if matriz.shape[0] != 120000:
            print(f"[ERROR] Clase #{i+1} no tiene 120000 filas → tiene {matriz.shape[0]}")
        if np.any(~np.isfinite(matriz)):
            print(f"[ERROR] Clase #{i+1} contiene NaN o infinitos.")

    X = np.hstack(matrices)
    print(f"[INFO] X final → shape = {X.shape} (esperado: (120000, 4))")
    return X


def aplicar_diferencia_finita(X):
    """
    Aplica diferencias finitas
    Returns:
        numpy.ndarray: X_diff ∈ ℜ^(119999, 4)
    """
    print(f"[INFO] aplicando diferencia finita → X.shape = {X.shape}")
    if X.shape != (120000, 4):
        print(f"[ERROR] X no tiene forma esperada (120000, 4)")
    X_diff = np.diff(X, axis=0)
    print(f"[INFO] X_diff.shape = {X_diff.shape} (esperado: (119999, 4))")
    return X_diff


def segmentar_muestras(X_diff, lF):
    """
    Segmenta muestras
    """
    filas = X_diff.shape[0]
    print(f"[INFO] segmentar_muestras: X_diff.shape = {X_diff.shape}, lF = {lF}")
    resto = filas % lF
    print(f"[INFO]  filas % lF = {filas} % {lF} = {resto}")

    if resto != 0:
        print(f"[WARN] {filas} no divisible por {lF}, se recortarán {resto} filas.")
        X_diff = X_diff[:filas - resto, :]

    n_segmentos = X_diff.shape[0] // lF
    print(f"[INFO] Se crearán {n_segmentos} segmentos de tamaño {lF}")
    segmentos = []
    for i in range(n_segmentos):
        start = i * lF
        end = (i + 1) * lF
        segmentos.append(X_diff[start:end, :])
    return segmentos


def calcular_entropias(segmentos, tipo_entropia, d, tau, c, S_max):
    """
    Calcula entropías multi-escala para los segmentos
    Args:
        segmentos (list): Lista de segmentos
        tipo_entropia (int): 1=MDE, 2=eMDE, 3=MPE, 4=eMPE
        d (int): Dimensión embedding
        tau (int): Factor de retardo
        c (int): Número de símbolos (solo para MDE/eMDE)
        S_max (int): Número máximo de escalas
    Returns:
        pandas.DataFrame: DataFrame con las entropías calculadas
    """
    # Validar parámetros (RNF-008)
    if not all(isinstance(p, int) and p > 0 for p in [d, tau, c, S_max]):
        raise ValueError("Todos los parámetros deben ser enteros positivos")
    features = []
    # Función para calcular entropías según el tipo
    def calcular_entropia_serie(serie):
        if tipo_entropia == 1:
            return entropy_mde(serie, d, tau, c, S_max)
        elif tipo_entropia == 2:
            return entropy_emde(serie, d, tau, c, S_max)
        elif tipo_entropia == 3:
            return entropy_mpe(serie, d, tau, S_max)
        elif tipo_entropia == 4:
            return entropy_empe(serie, d, tau, S_max)
        else:
            raise ValueError("tipo_entropia debe ser 1-4")
    # Procesar cada segmento y cada clase
    for segmento in segmentos:
        for clase in range(4):  # 4 clases
            serie = segmento[:, clase]
            entropias = calcular_entropia_serie(serie)
            features.append(entropias)
    return pd.DataFrame(features)

def generar_etiquetas(n_muestras_por_clase):
    """
    Genera etiquetas binarias para las clases
    Args:
        n_muestras_por_clase (int): Número de muestras por clase
    Returns:
        pandas.DataFrame: DataFrame con las etiquetas
    """
    etiquetas = []
    for i in range(4):
        etiquetas.extend([i] * n_muestras_por_clase)
    # Convertir a one-hot encoding
    one_hot = np.eye(4)[etiquetas]
    return pd.DataFrame(one_hot)

def guardar_configuracion(tipo_entropia, lF, d, tau, c, S_max):
    """
    Guarda la configuración en conf_temp.csv
    Args:
        tipo_entropia (int): 1-4
        lF (int): Longitud del segmento
        d (int): Dimensión embedding
        tau (int): Factor de retardo
        c (int): Número de símbolos
        S_max (int): Número máximo de escalas
    """
    # Validar parámetros (RNF-010)
    if not all(isinstance(p, int) and p > 0 for p in [tipo_entropia, lF, d, tau, c, S_max]):
        raise ValueError("Todos los parámetros deben ser enteros positivos")
    if tipo_entropia not in [1, 2, 3, 4]:
        raise ValueError("tipo_entropia debe ser 1-4")
    config = [tipo_entropia, lF, d, tau, c, S_max]
    pd.DataFrame(config).to_csv(CONF_TEMP, index=False, header=False)

def procesar_datos(tipo_entropia, lF, d, tau, c, S_max):
    print("\n[ETAPA] Iniciando preprocesamiento completo...\n")
    rutas = [os.path.join(DATA_DIR, f"Class{i+1}.csv") for i in range(4)]
    X = cargar_datos(rutas)
    X_diff = aplicar_diferencia_finita(X)
    segmentos = segmentar_muestras(X_diff, lF)
    print(f"[INFO] Total de segmentos generados = {len(segmentos)}")
    features = calcular_entropias(segmentos, tipo_entropia, d, tau, c, S_max)
    n_muestras = len(segmentos) * 4
    labels = generar_etiquetas(n_muestras // 4)
    guardar_configuracion(tipo_entropia, lF, d, tau, c, S_max)
    print("[INFO] Preprocesamiento finalizado exitosamente.\n")
    return features, labels


def guardar_resultados(features, labels):
    """
    Guarda los resultados en los archivos correspondientes
    Args:
        features (pandas.DataFrame): DataFrame con características
        labels (pandas.DataFrame): DataFrame con etiquetas
    """
    features.to_csv(os.path.join(DATA_DIR, "dClases.csv"), index=False, header=False)
    labels.to_csv(os.path.join(DATA_DIR, "dLabel.csv"), index=False, header=False)


In [33]:

def ppr():
    # leer conf
    conf = pd.read_csv(CONF_OPTIMO, header=None).values.flatten()
    tipo_entropia, lF, d, tau, c, S_max = map(int, conf)
    # procesar datos
    features, labels = procesar_datos(tipo_entropia, lF, d, tau, c, S_max)
    # guardar resultados
    guardar_resultados(features, labels)


In [34]:
ppr()


[ETAPA] Iniciando preprocesamiento completo...

[INFO] Cargando archivo: data/Class1.csv
[DEBUG] data/Class1.csv → shape = (120000, 4)
[INFO] Cargando archivo: data/Class2.csv
[DEBUG] data/Class2.csv → shape = (120000, 4)
[INFO] Cargando archivo: data/Class3.csv
[DEBUG] data/Class3.csv → shape = (120000, 4)
[INFO] Cargando archivo: data/Class4.csv
[DEBUG] data/Class4.csv → shape = (120000, 4)
[INFO] X final → shape = (120000, 4) (esperado: (120000, 4))
[INFO] aplicando diferencia finita → X.shape = (120000, 4)
[INFO] X_diff.shape = (119999, 4) (esperado: (119999, 4))
[INFO] segmentar_muestras: X_diff.shape = (119999, 4), lF = 1200
[INFO]  filas % lF = 119999 % 1200 = 1199
[WARN] 119999 no divisible por 1200, se recortarán 1199 filas.
[INFO] Se crearán 99 segmentos de tamaño 1200
[INFO] Total de segmentos generados = 99
[INFO] Preprocesamiento finalizado exitosamente.



# trn.py

In [50]:
"""
================================================================================
REQUERIMIENTOS FUNCIONALES Y NO FUNCIONALES: trn.py
(Etapa #2: Algoritmo de Entrenamiento)
================================================================================

--- RF-001: cargar_datos ---
| Campo               | Descripción                                                                 |
|---------------------|-----------------------------------------------------------------------------|
| **Descripción**     | Carga dClases.csv y dLabel.csv.                                             |
| **Entradas**        | - Rutas a dClases.csv y dLabel.csv.                                         |
| **Salidas**         | - Matrices 𝑋 (características) y 𝑦 (etiquetas).                             |
| **Parámetros**      | - ruta_X: Ruta a dClases.csv.                                               |
|                     | - ruta_y: Ruta a dLabel.csv.                                                |
| **Requerimientos NF**| [RNF-001] Validar mismo número de filas en 𝑋 y 𝑦.                          |
| **Ejemplo**         | X, y = cargar_datos("dClases.csv", "dLabel.csv")                            |

--- RF-002: reordenar_aleatoriamente ---
| Campo               | Descripción                                                                 |
|---------------------|-----------------------------------------------------------------------------|
| **Descripción**     | Reordena aleatoriamente 𝑋 y 𝑦 sincronizadamente.                            |
| **Entradas**        | - Matrices 𝑋 y 𝑦.                                                           |
| **Salidas**         | - Matrices 𝑋_shuffled y 𝑦_shuffled.                                         |
| **Parámetros**      | - seed: Semilla para reproducibilidad (ej. 42).                             |
| **Requerimientos NF**| [RNF-002] Usar np.random.seed(42).                                         |
|                     | [RNF-003] Validar orden sincronizado.                                       |
| **Ejemplo**         | X_shuffled, y_shuffled = reordenar_aleatoriamente(X, y)                     |

--- RF-003: normalizar_zscore ---
| Campo               | Descripción                                                                 |
|---------------------|-----------------------------------------------------------------------------|
| **Descripción**     | Normaliza características: 𝑥 = (𝑥 − mean(𝑥)) / std(𝑥).                       |
| **Entradas**        | - Matriz 𝑋.                                                                 |
| **Salidas**         | - Matriz 𝑋_norm normalizada.                                                |
| **Parámetros**      | - X: Matriz de características.                                             |
| **Requerimientos NF**| [RNF-004] Validar ausencia de divisiones por cero.                         |
| **Ejemplo**         | X_norm = normalizar_zscore(X)                                               |

--- RF-004: dividir_train_test ---
| Campo               | Descripción                                                                 |
|---------------------|-----------------------------------------------------------------------------|
| **Descripción**     | Divide datos en entrenamiento (65-80%) y prueba.                            |
| **Entradas**        | - 𝑋_norm, 𝑦_shuffled, porcentaje 𝑝 (65 < 𝑝 < 81).                            |
| **Salidas**         | - 𝑋_train, 𝑋_test, 𝑦_train, 𝑦_test.                                          |
| **Parámetros**      | - p: Porcentaje para entrenamiento (ej. 0.75).                              |
| **Requerimientos NF**| [RNF-005] Validar 𝑝 ∈ [0.65, 0.80].                                        |
| **Ejemplo**         | X_train, X_test, y_train, y_test = dividir_train_test(X_norm, y_shuffled, 0.75)|

--- RF-005: entrenar_mgd ---
| Campo               | Descripción                                                                 |
|---------------------|-----------------------------------------------------------------------------|
| **Descripción**     | Entrena modelo con mGD (descenso de gradiente con momentum).                |
| **Entradas**        | - 𝑋_train, 𝑦_train, max_iter, 𝜇, momentum.                                  |
| **Salidas**         | - Matriz de pesos 𝑊, vector de costo 𝐽.                                     |
| **Parámetros**      | - max_iter: Máximo de iteraciones.                                          |
|                     | - mu: Tasa de aprendizaje.                                                  |
|                     | - momentum: Factor de momentum.                                             |
| **Requerimientos NF**| [RNF-006] Inicializar 𝑊 aleatoriamente.                                    |
|                     | [RNF-007] Validar disminución monótona del costo.                           |
| **Ejemplo**         | W, J = entrenar_mgd(X_train, y_train, 1000, 0.01, 0.9)                      |

--- RF-006: guardar_pesos_y_costo ---
| Campo               | Descripción                                                                 |
|---------------------|-----------------------------------------------------------------------------|
| **Descripción**     | Guarda pesos y costos en pesos.csv y costo.csv.                             |
| **Entradas**        | - 𝑊, 𝐽, rutas de salida.                                                    |
| **Salidas**         | - Archivos pesos.csv y costo.csv.                                           |
| **Parámetros**      | - ruta_pesos: Ruta a pesos.csv.                                             |
|                     | - ruta_costo: Ruta a costo.csv.                                             |
| **Requerimientos NF**| [RNF-008] Validar dimensiones de 𝑊 y 𝐽.                                    |
| **Ejemplo**         | guardar_pesos_y_costo(W, J, "pesos.csv", "costo.csv")                       |

--- RF-007: cargar_configuracion ---
| Campo               | Descripción                                                                 |
|---------------------|-----------------------------------------------------------------------------|
| **Descripción**     | Carga configuración desde conf_train.csv.                                   |
| **Entradas**        | - Ruta a conf_train.csv.                                                    |
| **Salidas**         | - max_iter, 𝜇, 𝑝.                                                           |
| **Parámetros**      | - ruta: Ruta al archivo.                                                    |
| **Requerimientos NF**| [RNF-009] Validar valores numéricos positivos.                             |
| **Ejemplo**         | max_iter, mu, p = cargar_configuracion("conf_train.csv")                    |
"""
"""
trn.py - Algoritmo de Entrenamiento para Clasificación Multi-clases con Regresión Softmax

Este módulo implementa las funciones necesarias para:
1. Cargar y preprocesar datos de características y etiquetas
2. Normalizar características usando Z-score
3. Dividir datos en conjuntos de entrenamiento y prueba
4. Entrenar modelo usando descenso de gradiente con momentum (mGD)
5. Guardar pesos y costos del modelo

El archivo está diseñado para trabajar con configuraciones externas y no contiene
valores fijos, permitiendo optimización de parámetros desde main.py
"""

import os
import numpy as np
import pandas as pd
from typing import Tuple

# Constantes de rutas
DATA_DIR = "data"
CONF_DIR = "config"
CONF_TRAIN = os.path.join(CONF_DIR, "conf_train.csv")

def cargar_datos(ruta_X: str, ruta_y: str) -> Tuple[np.ndarray, np.ndarray]:
    """
    Carga datos de características y etiquetas desde archivos CSV

    Args:
        ruta_X: Ruta al archivo dClases.csv
        ruta_y: Ruta al archivo dLabel.csv

    Returns:
        X: Matriz de características (N x D)
        y: Matriz de etiquetas (N x K)

    Raises:
        ValueError: Si las dimensiones no coinciden o hay valores no finitos
    """
    X = pd.read_csv(ruta_X, header=None).values.astype(float)
    y = pd.read_csv(ruta_y, header=None).values.astype(float)

    # Validaciones (RNF-001)
    if X.shape[0] != y.shape[0]:
        raise ValueError(f"Número de filas diferente en X ({X.shape[0]}) e y ({y.shape[0]})")
    if not np.all(np.isfinite(X)):
        raise ValueError("X contiene valores no finitos")
    if not np.all(np.isfinite(y)):
        raise ValueError("y contiene valores no finitos")

    return X, y

def reordenar_aleatoriamente(X: np.ndarray, y: np.ndarray, seed: int = 42) -> Tuple[np.ndarray, np.ndarray]:
    """
    Reordena aleatoriamente X y y de manera sincronizada

    Args:
        X: Matriz de características
        y: Matriz de etiquetas
        seed: Semilla para reproducibilidad

    Returns:
        X_shuffled: Matriz de características reordenada
        y_shuffled: Matriz de etiquetas reordenada
    """
    # Validación (RNF-002, RNF-003)
    if X.shape[0] != y.shape[0]:
        raise ValueError("X e y deben tener el mismo número de filas")

    np.random.seed(seed)
    perm = np.random.permutation(X.shape[0])
    return X[perm], y[perm]

def normalizar_zscore(X: np.ndarray) -> np.ndarray:
    """
    Normaliza características usando Z-score: x = (x - mean(x)) / std(x)

    Args:
        X: Matriz de características

    Returns:
        X_norm: Matriz normalizada

    Raises:
        ValueError: Si hay división por cero
    """
    # Validación (RNF-004)
    if X.size == 0:
        return X

    mean = X.mean(axis=0)
    std = X.std(axis=0)

    if np.any(std == 0):
        raise ValueError("Desviación estándar cero detectada - no se puede normalizar")

    return (X - mean) / std

def dividir_train_test(X_norm: np.ndarray, y_shuffled: np.ndarray, p: float) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    """
    Divide datos en conjuntos de entrenamiento y prueba

    Args:
        X_norm: Matriz de características normalizada
        y_shuffled: Matriz de etiquetas reordenada
        p: Porcentaje para entrenamiento (0.65 < p < 0.80)

    Returns:
        X_train, X_test, y_train, y_test

    Raises:
        ValueError: Si p no está en el rango válido
    """
    # Validación (RNF-005)
    if not (0.65 <= p <= 0.80):
        raise ValueError("p debe estar entre 0.65 y 0.80")

    N = X_norm.shape[0]
    L = int(np.round(N * p))
    L = max(1, min(L, N - 1))  # Asegurar al menos una muestra en cada conjunto

    return X_norm[:L], X_norm[L:], y_shuffled[:L], y_shuffled[L:]

def softmax(z: np.ndarray) -> np.ndarray:
    """
    Calcula la función softmax de manera numéricamente estable

    Args:
        z: Matriz de scores lineales

    Returns:
        Matriz de probabilidades
    """
    exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))
    return exp_z / np.sum(exp_z, axis=1, keepdims=True)
def entrenar_mgd(X_train: np.ndarray, y_train: np.ndarray, max_iter: int, mu: float, beta: float = 0.9) -> Tuple[np.ndarray, np.ndarray]:
    """
    Entrena modelo usando descenso de gradiente con momentum (mGD)
    """
    print("\n[ETAPA] Entrenamiento con mGD")
    print(f"[INFO] X_train.shape = {X_train.shape}")
    print(f"[INFO] y_train.shape = {y_train.shape}")
    print(f"[INFO] max_iter = {max_iter}, mu = {mu}, beta = {beta}")

    if X_train.shape[0] != y_train.shape[0]:
        raise ValueError("X_train y y_train deben tener el mismo número de muestras")

    D = X_train.shape[1]
    K = y_train.shape[1]

    print(f"[INFO] D (features) = {D}, K (clases) = {K}")

    # Inicializar pesos
    W = np.random.randn(D, K) * 0.01  # W inicial sin bias
    print(f"[INFO] W inicial (sin bias) shape = {W.shape}")

    # Agregar bias a entrada y pesos
    Xb = np.hstack([X_train, np.ones((X_train.shape[0], 1))])     # Bias como columna extra en X
    W = np.vstack([W, np.zeros((1, K))])                           # Bias como fila extra en W
    print(f"[INFO] Xb shape (con bias) = {Xb.shape}")
    print(f"[INFO] W shape (con bias) = {W.shape}")

    V = np.zeros_like(W)
    J = np.zeros(max_iter)
    eps = 1e-15

    for it in range(max_iter):
        z = Xb @ W
        yhat = softmax(z)

        J[it] = -np.mean(np.sum(y_train * np.log(yhat + eps), axis=1))

        grad = (Xb.T @ (yhat - y_train)) / X_train.shape[0]
        V = beta * V + mu * grad
        W = W - V
        if it == 0 or (it + 1) % (max_iter // 5) == 0 or it == max_iter - 1:
            preds = np.argmax(yhat, axis=1)
            clases, counts = np.unique(preds, return_counts=True)
            print(f"[DEBUG] Predicciones por clase: {dict(zip(clases, counts))}")
    
            print(f"[INFO] Iteración {it+1}/{max_iter} → Costo J = {J[it]:.6f}")

    print("[ok] Entrenamiento finalizado.\n")
    return W, J


def guardar_pesos_y_costo(W: np.ndarray, J: np.ndarray, ruta_pesos: str, ruta_costo: str) -> None:
    """
    Guarda pesos y costos en archivos CSV

    Args:
        W: Matriz de pesos
        J: Vector de costos
        ruta_pesos: Ruta para guardar pesos
        ruta_costo: Ruta para guardar costos

    Raises:
        ValueError: Si las dimensiones no son válidas
    """
    # Validación (RNF-008)
    if W.ndim != 2:
        raise ValueError("W debe ser una matriz 2D")
    if J.ndim != 1:
        raise ValueError("J debe ser un vector 1D")

    os.makedirs(DATA_DIR, exist_ok=True)
    pd.DataFrame(W).to_csv(ruta_pesos, index=False, header=False)
    pd.DataFrame(J).to_csv(ruta_costo, index=False, header=False)

def cargar_configuracion(ruta: str = CONF_TRAIN) -> Tuple[int, float, float]:
    """
    Carga configuración desde archivo conf_train.csv

    Args:
        ruta: Ruta al archivo de configuración

    Returns:
        max_iter: Número máximo de iteraciones
        mu: Tasa de aprendizaje
        p: Porcentaje para entrenamiento

    Raises:
        ValueError: Si los valores no son válidos
    """
    conf = pd.read_csv(ruta, header=None).values

    # Validaciones (RNF-009)
    max_iter = int(conf[0, 0])
    mu = float(conf[1, 0])
    p = float(conf[2, 0]) / 100.0 if conf[2, 0] > 1 else conf[2, 0]

    if not (0 < mu < 1):
        raise ValueError("mu debe estar entre 0 y 1")
    if not (0.65 <= p <= 0.80):
        raise ValueError("p debe estar entre 0.65 y 0.80")

    return max_iter, mu, p



In [51]:

def trn():
    """
    Función principal para ejecutar el proceso de entrenamiento
    """
    # Cargar configuración
    max_iter, mu, p = cargar_configuracion()

    # Cargar datos
    X, y = cargar_datos(os.path.join(DATA_DIR, "dClases.csv"), 
                       os.path.join(DATA_DIR, "dLabel.csv"))

    # Reordenar aleatoriamente
    X_shuffled, y_shuffled = reordenar_aleatoriamente(X, y)

    # Normalizar
    X_norm = normalizar_zscore(X_shuffled)

    # Dividir en train/test
    X_train, X_test, y_train, y_test = dividir_train_test(X_norm, y_shuffled, p)

    # Guardar conjuntos de entrenamiento y prueba
    pd.DataFrame(X_train).to_csv(os.path.join(DATA_DIR, "dtrn.csv"), index=False, header=False)
    pd.DataFrame(y_train).to_csv(os.path.join(DATA_DIR, "dtrn_label.csv"), index=False, header=False)
    pd.DataFrame(X_test).to_csv(os.path.join(DATA_DIR, "dtst.csv"), index=False, header=False)
    pd.DataFrame(y_test).to_csv(os.path.join(DATA_DIR, "dtst_label.csv"), index=False, header=False)

    # Entrenar modelo
    W, J = entrenar_mgd(X_train, y_train, max_iter, mu)

    # Guardar resultados
    guardar_pesos_y_costo(W, J,
                         os.path.join(DATA_DIR, "pesos.csv"),
                         os.path.join(DATA_DIR, "costo.csv"))

    print("[ok] Entrenamiento finalizado. Archivos guardados en data/")



In [52]:
trn()



[ETAPA] Entrenamiento con mGD
[INFO] X_train.shape = (317, 10)
[INFO] y_train.shape = (317, 4)
[INFO] max_iter = 10000, mu = 0.2, beta = 0.9
[INFO] D (features) = 10, K (clases) = 4
[INFO] W inicial (sin bias) shape = (10, 4)
[INFO] Xb shape (con bias) = (317, 11)
[INFO] W shape (con bias) = (11, 4)
[DEBUG] Predicciones por clase: {np.int64(0): np.int64(76), np.int64(1): np.int64(47), np.int64(2): np.int64(100), np.int64(3): np.int64(94)}
[INFO] Iteración 1/10000 → Costo J = 1.386166
[DEBUG] Predicciones por clase: {np.int64(0): np.int64(89), np.int64(1): np.int64(79), np.int64(2): np.int64(91), np.int64(3): np.int64(58)}
[INFO] Iteración 2000/10000 → Costo J = 1.325039
[DEBUG] Predicciones por clase: {np.int64(0): np.int64(89), np.int64(1): np.int64(79), np.int64(2): np.int64(91), np.int64(3): np.int64(58)}
[INFO] Iteración 4000/10000 → Costo J = 1.325039
[DEBUG] Predicciones por clase: {np.int64(0): np.int64(89), np.int64(1): np.int64(79), np.int64(2): np.int64(91), np.int64(3): np.

# test.py

In [ ]:
# Testing for Softmax Regression
import numpy as np
import os
import pandas as pd

DATA_DIR = "data"

def cargar_datos(ruta_X_test, ruta_y_test, ruta_W):
    print("[ETAPA] Cargando datos de prueba y pesos...")
    X_test = pd.read_csv(ruta_X_test, header=None).values.astype(float)
    y_test = pd.read_csv(ruta_y_test, header=None).values.astype(float)
    W = pd.read_csv(ruta_W, header=None).values.astype(float)

    print(f"[INFO] X_test shape = {X_test.shape}")
    print(f"[INFO] y_test shape = {y_test.shape}")
    print(f"[INFO] W shape = {W.shape}")

    if X_test.shape[0] != y_test.shape[0]:
        raise ValueError("X_test y y_test deben tener el mismo número de filas")
    return X_test, y_test, W

def predecir_softmax(X_test, W):
    print("[ETAPA] Calculando probabilidades con softmax...")
    Xb = np.hstack([X_test, np.ones((X_test.shape[0], 1))])
    z = Xb @ W
    exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))
    y_pred_proba = exp_z / np.sum(exp_z, axis=1, keepdims=True)
    return y_pred_proba

def obtener_etiquetas_predichas(y_pred_proba):
    return np.argmax(y_pred_proba, axis=1)

def matriz_confusion(y_true, y_pred):
    print("[ETAPA] Calculando matriz de confusión...")
    cm = np.zeros((4, 4), dtype=int)
    for i in range(4):
        for j in range(4):
            cm[i, j] = np.sum((y_true == i) & (y_pred == j))
    print("[INFO] Matriz de confusión:\n", cm)
    return cm

def calcular_fscores(cm):
    print("[ETAPA] Calculando F-scores por clase...")
    fscores = np.zeros(4)
    for i in range(4):
        TP = cm[i, i]
        FP = np.sum(cm[:, i]) - TP
        FN = np.sum(cm[i, :]) - TP
        precision = TP / (TP + FP) if (TP + FP) > 0 else 0.0
        recall = TP / (TP + FN) if (TP + FN) > 0 else 0.0
        fscores[i] = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0
        print(f"[INFO] Clase {i}: Precision = {precision:.4f}, Recall = {recall:.4f}, F-score = {fscores[i]:.4f}")
    print(f"[INFO] Macro F-score promedio = {np.mean(fscores):.4f}")
    return fscores

def guardar_resultados(cm, fscores, ruta_cm, ruta_fs):
    os.makedirs(DATA_DIR, exist_ok=True)
    pd.DataFrame(cm).to_csv(ruta_cm, index=False, header=False)
    pd.DataFrame(fscores).to_csv(ruta_fs, index=False, header=False)
    print(f"[ok] Resultados guardados en:\n - {ruta_cm}\n - {ruta_fs}")


In [62]:

def test():
    print("\n========================")
    print("   EVALUACIÓN DEL MODELO")
    print("========================\n")

    # Cargar datos
    X_test, y_test, W = cargar_datos(
        os.path.join(DATA_DIR, "dtst.csv"),
        os.path.join(DATA_DIR, "dtst_label.csv"),
        os.path.join(DATA_DIR, "pesos.csv")
    )

    # Convertir etiquetas one-hot a clases
    y_test_labels = np.argmax(y_test, axis=1)

    # Predicción
    y_pred_proba = predecir_softmax(X_test, W)
    y_pred = obtener_etiquetas_predichas(y_pred_proba)

    # Matriz de confusión y F-scores
    cm = matriz_confusion(y_test_labels, y_pred)
    fscores = calcular_fscores(cm)

    # Guardar resultados
    guardar_resultados(
        cm, fscores,
        os.path.join(DATA_DIR, "cmatriz.csv"),
        os.path.join(DATA_DIR, "fscores.csv")
    )

    print("\n[ok] Evaluación finalizada.\n")


In [63]:
test()




   EVALUACIÓN DEL MODELO

[ETAPA] Cargando datos de prueba y pesos...
[INFO] X_test shape = (79, 10)
[INFO] y_test shape = (79, 4)
[INFO] W shape = (11, 4)
[ETAPA] Calculando probabilidades con softmax...
[ETAPA] Calculando matriz de confusión...
[INFO] Matriz de confusión:
 [[6 4 4 4]
 [6 4 5 4]
 [6 3 5 6]
 [6 6 6 4]]
[ETAPA] Calculando F-scores por clase...
[INFO] Clase 0: Precision = 0.2500, Recall = 0.3333, F-score = 0.2857
[INFO] Clase 1: Precision = 0.2353, Recall = 0.2105, F-score = 0.2222
[INFO] Clase 2: Precision = 0.2500, Recall = 0.2500, F-score = 0.2500
[INFO] Clase 3: Precision = 0.2222, Recall = 0.1818, F-score = 0.2000
[INFO] Macro F-score promedio = 0.2395
[ok] Resultados guardados en:
 - data/cmatriz.csv
 - data/fscores.csv

[ok] Evaluación finalizada.



# main.py